<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Teste_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
! pip install keras==2.7.0
! pip show keras

     |████████████████████████████████| 1.3 MB 18.6 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.
Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: tensorflow, keras-vis


In [34]:
import pandas as pd
import numpy as np
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [35]:
import pandas as pd
import numpy
from numpy import array
from numpy import asarray
from numpy import zeros

In [36]:
dfx = pd.read_csv('/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv')
dfx.head(2)

,Unnamed: 0,text,label
0,1,retwet eu queria compartilhar mas antes eu gos...,not_hate
1,2,gostei da pulseira e da educação dele nossaaa ...,not_hate


In [37]:
palavras_unicas = list(dfx['text'].str.split(' ', expand=True).stack().unique())
print(len(palavras_unicas))
print(palavras_unicas)

51970
['retwet', 'eu', 'queria', 'compartilhar', 'mas', 'antes', 'gostaria', 'de', 'saber', 'o', 'motivo', 'dessa', 'briga', 'toda', 'alguém', 'pode', 'me', 'dizer', 'gostei', 'da', 'pulseira', 'e', 'educação', 'dele', 'nossaaa', 'muita', 'elegância', 'mesmo', 'limpa', 'fi', 'esses', 'não', 'irão', 'ao', 'encontro', 'beijing', 'cadê', 'eles', 'está', 'faltando', 'questionando', 'quem', 'é', 'esse', 'blogueiro', 'sem', 'blog', 'estamos', 'juntos', 'renan', 'caloteiro', 'desonesto', 'vejam', 'link', 'abaixo', 'só', 'nós', 'já', 'entendemos', 'tudo', 'isso', 'as', 'vezes', 'penso', 'que', 'seria', 'melhor', 'bolsonaro', 'se', 'afastar', 'entregar', 'comando', 'para', 'morão', 'capitão', 'fez', 'a', 'parte', 'seremos', 'sempre', 'gratos', 'barco', 'deriva', 'precisamos', 'um', 'pulso', 'mais', 'forte', 'p', 'segurar', 'leme', 'demorou', 'cara', 'bom', 'bota', 'gente', 'assim', 'bahia', 'em', 'vai', 'fazer', 'anos', 'sendo', 'governada', 'pelo', 'pt', 'sei', 'como', 'baiano', 'vem', 'sup', 

In [38]:
# with open('/content/drive/MyDrive/TCC/dados/processadas/Base1_palavras_unicas.txt', 'w') as f:
#     for item in palavras_unicas:
#         f.write("%s\n" % item)

# Filtra os embeddings

In [39]:
# g = open('./embeddings/baseFinal_so_label_texto_palavras_unicas.txt', encoding='utf-8')
# embeddings_de_interesse = list()
# contador = 0
# for line in g:
#     values = line.split()
#     word = values[0]
#     embeddings_de_interesse.append(word)
# g.close()

# print(embeddings_de_interesse)

In [40]:

# Código para filtrar o arquivo de embeddings
# gerando um novo arquivo só com as palavras desejadas.

embeddings_index = dict()
f = open('/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s50_filtrado.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    if (word in palavras_unicas):
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()


In [41]:
print(embeddings_index)

{'de': array([-3.9998 , -6.0781 , -0.53603,  1.2633 , -1.6086 , -2.1699 ,
       -1.794  ,  0.98339, -0.96582, -4.5025 ,  0.48768, -1.427  ,
        2.8924 ,  1.3856 , -0.9774 , -1.0603 ,  1.5051 ,  2.4258 ,
        2.8143 ,  1.2687 , -0.19178,  3.1669 ,  2.0471 ,  1.666  ,
       -1.767  ,  2.9318 , -1.5953 , -1.2464 ,  1.3831 ,  5.3079 ,
        1.8239 , -2.5159 ,  2.6014 , -0.55257,  0.94082, -0.33999,
       -2.0699 ,  0.58165, -0.57305, -0.53186, -1.6609 , -1.3734 ,
       -0.10354, -0.58749,  1.3886 , -0.55111, -0.88969,  0.70573,
        2.9843 ,  0.42119], dtype=float32), 'a': array([-3.9998 , -6.0781 , -0.53603,  1.2633 , -1.6086 , -2.1699 ,
       -1.794  ,  0.98339, -0.96582, -4.5025 ,  0.48768, -1.427  ,
        2.8924 ,  1.3856 , -0.9774 , -1.0603 ,  1.5051 ,  2.4258 ,
        2.8143 ,  1.2687 , -0.19178,  3.1669 ,  2.0471 ,  1.666  ,
       -1.767  ,  2.9318 , -1.5953 , -1.2464 ,  1.3831 ,  5.3079 ,
        1.8239 , -2.5159 ,  2.6014 , -0.55257,  0.94082, -0.33999,
      

In [42]:
print(len(embeddings_index.keys()))

4349


In [43]:
def monta_linha(palavra, coeficientes):
    # Monta a linha com palavra _ coeficientes.
    
    linha = f'{palavra}'
    for coef in coeficientes:
        linha += f' {coef:.5f}'
    linha += '\n'
    
    return linha
    

In [44]:
# Grava o novo arquivo com as embeddings de interesse.

f = open('/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s50_filtrado.txt', 'w', encoding='utf-8')
# imprime a linha de cabecalho: 
cabecalho = f"{len(embeddings_index)} {len(embeddings_index['a'])}\n"
f.write(cabecalho)
for word in embeddings_index:
    coeficientes = embeddings_index[word]
    linha = monta_linha(word, coeficientes)
    f.write(linha)
f.close()


## fim da rotina

In [45]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s50_filtrado.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [46]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(palavras_unicas)
#word_index: dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.
word_index = token.word_index

In [47]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [49]:
print(type(embedding_matrix))

<class 'numpy.ndarray'>


In [ ]:
https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/#:~:text=You%20can%20save%20your%20NumPy,file%2C%20most%20commonly%20a%20comma.